# Lab 2: Characteristics of remotely sensed data

In [1]:
from datetime import date
today = date.today()
d2 = today.strftime("%B %d, %Y")
print("Updated by Alfonso Torres-Rua, ", d2)

Updated by Alfonso Torres-Rua,  February 21, 2020


## **Purpose:**
The purpose of this lab is to demonstrate concepts of spatial, spectral, temporal and radiometric resolution.  You will be introduced to image data from several sensors aboard various platforms.  At the completion of the lab, you will be able to understand the difference between remotely sensed datasets based on sensor characteristics and how to choose an appropriate remotely sensed dataset based on these concepts.  

**Prerequisites:** Lab 1

## 1. Temporal Resolution

Every satellite is different from each other, spectrally, radiometrically, and temporally. This is due to multiple reasons such as access to technology, monetary resources, application targets, mission duration, and others. See these videos for two satellites:

- Landsat orbit:
https://svs.gsfc.nasa.gov//vis/a010000/a011400/a011481/G2014-016_LDCM_orbit_MASTER_ipod_lg.m4v

- MODIS orbit: https://svs.gsfc.nasa.gov//vis/a000000/a003300/a003348/aquacomp720.mp4

## 2. Spatial resolution

In the present context, spatial resolution means pixel size.  In practice, spatial resolution depends on the projection of the sensor's instantaneous field of view (IFOV) on the ground and how a set of radiometric measurements are resampled into a regular grid.  To see the difference in spatial resolution resulting from different sensors, visualize data at different scales from different sensors.

In [2]:
# Initializing display and earthengine
from IPython.display import Image
%matplotlib inline

import ee
ee.Initialize()

# importing ipygee for dynamic mapping
from ipygee import *

Map = Map() # from ipygee
Map.show()

Map(center=[0, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text'…

### - MODIS.  
There are two Moderate Resolution Imaging Spectro-Radiometers ([MODIS](http://modis.gsfc.nasa.gov/)) aboard the [Terra](http://terra.nasa.gov/) and [Aqua](http://aqua.nasa.gov/) satellites.  Different MODIS [bands](http://modis.gsfc.nasa.gov/about/specifications.php) produce data at different spatial resolutions.  For the visible bands, the lowest common resolution is 500 meters (red and NIR are 250 meters).  

Search for 'MYD09GA' in the [Google EarthEngine datasets](https://developers.google.com/earth-engine/datasets/catalog/) and retrieve (copy) the path to 'MYD09GA.006 Aqua Surface Reflectance Daily Global 1km and 500m'.  Name the import myd09. ([Complete list of MODIS land products](https://lpdaac.usgs.gov/dataset_discovery/modis/modis_products_table).  Note that Terra MODIS datasets start with 'MOD' and MODIS Aqua datasets start with 'MYD').

In [3]:
myd09 = ee.ImageCollection("MODIS/006/MYD09GA")

# Define a location of interest as a point at SFO airport.
# from geopy.geocoders import Nominatim
# geolocator = Nominatim(user_agent="a.torres@aggiemail.usu.edu") #put your email here
# address, (latitude, longitude) = geolocator.geocode("San Francisco")
# print(latitude, longitude)


latitude =37.77862975352838
longitude = -122.22365888039644
sfoPoint = ee.Geometry.Point([longitude, latitude]);

#Define a region to display (from the entire planet to a small region). USE THE COORDINATES FROM GOOGLE EARTH PRO
# area_to_display = ee.Geometry.Rectangle([-179, 80, 179, -80]) #entire planet

To display a false-color MODIS image, select an image acquired by the Aqua MODIS sensor and display it for SFO:

In [4]:
# Get a surface reflectance image from the MODIS MYD09GA collection.
modisImage = ee.Image(myd09.filterDate('2019-07-06').first());

modisImage = modisImage.multiply(0.0001)
# Print the image band names to the console.
band_names = modisImage.bandNames()
print(band_names.getInfo())

['num_observations_1km', 'state_1km', 'SensorZenith', 'SensorAzimuth', 'Range', 'SolarZenith', 'SolarAzimuth', 'gflags', 'orbit_pnt', 'granule_pnt', 'num_observations_500m', 'sur_refl_b01', 'sur_refl_b02', 'sur_refl_b03', 'sur_refl_b04', 'sur_refl_b05', 'sur_refl_b06', 'sur_refl_b07', 'QC_500m', 'obscov_500m', 'iobs_res', 'q_scan']


These names are described also here: [modis bands](https://modis.gsfc.nasa.gov/about/specifications.php)

In [5]:
# Use these MODIS bands for red, green, blue, respectively.
modisBands = ['sur_refl_b01', 'sur_refl_b04', 'sur_refl_b03'];

#// Add the image to the map.
# url = modisImage.getThumbURL({'dimensions': 700,'bands':modisBands,'min': 0, 'max': 3000,'region':area_to_display})
# # print(url)
# Image(url=url)

# add layer
Map.addLayer(modisImage, {'bands':modisBands,'min': 0, 'max': 1}, name='MODIS Image')

Note the size of pixels with respect to objects on the ground.  Print the size of the pixels (in meters) with:


In [6]:
# Get the scale of the data from the first band's projection:
modisScale = modisImage.select('sur_refl_b01').projection().nominalScale();

print('MODIS scale:', modisScale.getInfo());


MODIS scale: 463.3127165275


In [7]:
# add layer SensorZenith
Map.addLayer(modisImage, {'bands':'SensorZenith','min': 0, 'max': 0.3}, name='MODIS SensorZenith')

It's also worth noting that these MYD09 data are surface reflectance scaled by 10000 (not TOA reflectance), meaning that clever NASA scientists have done a fancy atmospheric correction for you!


## - Landsat MSS.  

Multi-spectral scanners (MSS) were flown aboard Landsats 1-5.  MSS data have a spatial resolution of 60 meters.
Search for 'landsat 5 mss' and import the result called 'USGS Landsat 5 MSS Collection 1 Tier 2 Raw Scenes'.  Name the import mss.

To visualize MSS data over SFO, for a relatively less cloudy image, use:


In [8]:
# Filter MSS imagery by location, date and cloudiness.
mss= ee.ImageCollection("LANDSAT/LM05/C01/T2")

mssImage = ee.Image(mss
    .filterBounds(sfoPoint)
    .sort('CLOUD_COVER')
#Get the least cloudy image.
    .first());

# Print the image band names to the console.
band_names = mssImage.bandNames()
print(band_names.getInfo())
# Display the MSS image as a color-IR composite.


['B1', 'B2', 'B3', 'B4', 'BQA']


In [9]:
# Use these MODIS bands for red, green, blue, respectively.
mssBands = ['B3', 'B2', 'B1'];

#// Add the image to the map.
# url = mssImage.getThumbURL({'dimensions': 700,'bands':mssBands,'min': 0, 'max': 200}) # because the area is small we don't need region
# # print(url)
# Image(url=url)

Map.addLayer(mssImage, {'bands':mssBands,'min': 0, 'max': 200}, name='LandsatMSS Image')

bounds = sfoPoint.buffer(70000)
Map.centerObject(bounds)

Let's sample MODIS scene to Landsat scale. Let's use the file [Landsat WRS 2 Descending Path Row Shapefile](https://www.usgs.gov/media/files/landsat-wrs-2-descending-path-row-shapefile). Download it, unzip it and visualize it in Google Earth pro. There is a copy of the file in this repo too.

In [10]:
# landsat_area = ee.Geometry.Rectangle([-123.4, 38.5,-120.7, 36.5]) #Define upper left, lower right corner of Landsat image over san Francisco.
# #// Add the image to the map.
# modisBands = ['sur_refl_b02', 'sur_refl_b01', 'sur_refl_b04'];

# url = modisImage.getThumbURL({'dimensions': 700,'bands':modisBands,'min': 0, 'max': 6000,'region':landsat_area})
# # print(url)
# Image(url=url)

Check the scale (in meters) as previously:

In [11]:
# Get the scale of the MSS data from its projection:
mssScale = mssImage.select('B1').projection().nominalScale();

print('MSS scale:', mssScale.getInfo());


MSS scale: 60


## -TM.  

The Thematic Mapper ([TM](http://landsat.gsfc.nasa.gov/?p=3229)) was flown aboard Landsats 4-5.  (It was succeeded by the Enhanced Thematic Mapper ([ETM+](http://landsat.gsfc.nasa.gov/?p=3225)) aboard Landsat 7 and the Operational Land Imager ([OLI](http://landsat.gsfc.nasa.gov/?p=5447)) / Thermal Infrared Sensor ([TIRS](http://landsat.gsfc.nasa.gov/?p=5474)) sensors aboard Landsat 8.)  TM data have a spatial resolution of 30 meters.

1. Search for 'landsat 5 toa' and import the first result (which should be 'USGS Landsat 5 TM Collection 1 Tier 1 TOA Reflectance'.  Name the import tm.

2. Visualize MSS data over SFO, for approximately the same area as the MSS image, use  image.geometry():


In [12]:
tm= ee.ImageCollection("LANDSAT/LT05/C01/T1_TOA")

# // Filter TM imagery by location, date and cloudiness.
tmImage = ee.Image(tm
    .filterBounds(sfoPoint) # I could have used sfopoint too. every image has a boundary geometry
    .filterDate('2011-05-01', '2011-10-01')
    .sort('CLOUD_COVER') # not all satellites have this property, check the properties table
    .first());

# Print the image band names to the console.
band_names = tmImage.bandNames()
print(band_names.getInfo())

['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'BQA']


In [13]:
# // Display the TM image as a color-IR composite.
tmBands=  ['B4', 'B3', 'B2']
#// Add the image to the map.
# url = tmImage.getThumbURL({'dimensions': 700,'bands':tmBands,'min': 0, 'max': 0.4}) # because the area is small we don't need region
# # print(url)
# Image(url=url)

Map.addLayer(tmImage, {'bands':tmBands,'min': 0, 'max': 0.4}, name='Landsat TM Image')

bounds = sfoPoint.buffer(70000)
Map.centerObject(bounds)

Check the scale (in meters) as previously:

In [14]:
# // Get the scale of the TM data from its projection:
tmScale = tmImage.select('B1').projection().nominalScale();
print('TM scale:', tmScale.getInfo());


TM scale: 30


In [15]:
## - Sentinel 2 

## -NAIP.  
The National Agriculture Imagery Program ([NAIP](http://www.fsa.usda.gov/programs-and-services/aerial-photography/imagery-programs/naip-imagery/)) is an effort to acquire imagery over the continental US on a 3-year rotation using airborne sensors.  The imagery have a spatial resolution of 1-2 meters.  

1. Search for 'naip' and import the first result (which should be 'NAIP: National Agriculture Imagery Program'.  Name the import naip.

2. Since NAIP imagery is distributed as quarters of Digital Ortho Quads (DOQs) at irregular cadence, load everything from the most recent year in the acquisition cycle (2012) over the study area and mosaic() it:


In [16]:
naip = ee.ImageCollection("USDA/NAIP/DOQQ")

# // Get NAIP images for the study period and region of interest.
naipImages = naip.filterDate('2012-01-01', '2012-12-31').filterBounds(sfoPoint);

# // Mosaic adjacent images into a single image.
naipImage = naipImages.mosaic();

# Print the image band names to the console.
band_names = naipImage.bandNames()
print(band_names.getInfo())


['R', 'G', 'B', 'N']


In [17]:
#Define a region to display (from the entire planet to a small region). USE THE COORDINATES FROM GOOGLE EARTH PRO
# area_to_display = ee.Geometry.Rectangle([-122.5, 37.8, -122.44, 37.75])

# // Display the NAIP mosaic as a color-IR composite.
nBands=  ['N', 'R', 'G']
#// Add the image to the map.
# url = naipImage.getThumbURL({'dimensions': 700,'bands':nBands,'min': 0, 'max': 255, 'region':area_to_display}) # because the area is small we don't need region
# # print(url)
# Image(url=url)

Map.addLayer(naipImage, {'bands':nBands,'min': 0, 'max': 255}, name='NAIP Image')

bounds = sfoPoint.buffer(70000)
Map.centerObject(bounds)

In [18]:
# // Get the NAIP resolution from the first image in the mosaic.
naipScale = ee.Image(naipImages.first()).projection().nominalScale();
print('NAIP scale:', naipScale.getInfo());


NAIP scale: 1


## Assignment
1. What are the necessary changes to this notebook, to perform the same steps for Cache Valley? Produce a notebook.

2. There are more satellites and data sources available to you. To your Logan notebook add the following satellites completing the same steps we did for each of the imagery sources in this notebook.

The imagery sources to add are:

ASTER: ee.ImageCollection("ASTER/AST_L1T_003"), hint: request 2019 growing season
https://developers.google.com/earth-engine/datasets/catalog/ASTER_AST_L1T_003

Sentinel-2: ee.ImageCollection('COPERNICUS/S2')
https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2

Sentinel-3: ee.ImageCollection("COPERNICUS/S3/OLCI") hint: this satellite behaves like MODIS
https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S3_OLCI

PROBA-V: ee.ImageCollection("VITO/PROBAV/C1/S1_TOC_100M") hint: request 2019 gorwing season
https://developers.google.com/earth-engine/datasets/catalog/VITO_PROBAV_C1_S1_TOC_100M

VIIRS: ee.ImageCollection("NOAA/VIIRS/001/VNP09GA") hint: this satellite behaves like MODIS
https://developers.google.com/earth-engine/datasets/catalog/NOAA_VIIRS_001_VNP09GA

BONUS: HYPERSPECTRAL SENSOR
EO-1 Hyperion Hyperspectral Imager: ee.ImageCollection("EO1/HYPERION") 
hint: this satellite has 220 bands, choose between 2001 and 2017 to get an image for Cache Valley
https://developers.google.com/earth-engine/datasets/catalog/EO1_HYPERION


Happy coding!